1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

        *Наименование вакансии
        *Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
        *Ссылку на саму вакансию        
        *Сайт откуда собрана вакансия

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

In [2]:
%config IPCompleter.greedy=True

In [9]:
vacancy_name = input('Введите вакансию:')

Введите вакансию:Аналитик


In [10]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'}
link = 'https://hh.ru/search/vacancy'

In [14]:
def get_pages():
    columns = ['VacancyName', 'Link', 'Min Rate', 'Max Rate']
    df = pd.DataFrame(columns=columns)
    r = requests.get(f'{link}??L_is_autosearch=false&clusters=true&enable_snippets=true&text=\
    {vacancy_name}&page=0', headers=headers)
    soup = bs(r.text, 'html.parser')
    max_pages = int(max([a['data-page'] for a in soup.find_all('a', {'class': 'bloko-button HH-Pager-Control'})]))
    if max_pages > 1:
        for i in range(max_pages):
            r = requests.get(f'{link}??L_is_autosearch=false&clusters=true&enable_snippets=true&text=\
            {vacancy_name}&page={i}', headers=headers)
            soup = bs(r.text, 'html.parser')
            for j in range(len(soup.find_all('a', {'class': 'bloko-link HH-LinkModifier'}))):

                name = soup.find_all('a', {'class': 'bloko-link HH-LinkModifier'})[j].string

                link2 = [a['href'] for a in soup.find_all('a', {'class': 'bloko-link HH-LinkModifier'}, href=True)][j] \
                    .replace(f'?query={vacancy_name}', '')

                all_tags = soup.find_all('div', {'class': 'vacancy-serp-item__row vacancy-serp-item__row_header'})[j]

                sal = all_tags.find_all('div', {'class': 'vacancy-serp-item__compensation'})

                if len(all_tags.find_all('div', {'class': 'vacancy-serp-item__compensation'})) == 0:
                    continue
                if (len(sal) == 1) and (len(re.findall(r'[-]', sal[0].string)) == 0):
                    if sal[0].string.replace(u'\xa0', '').split(' ', 1)[0] == 'от':
                        min_rate = sal[0].string.replace(u'\xa0', '').split(' ', 1)[1]
                        max_rate = ''
                    else:
                        min_rate = ''
                        max_rate = sal[0].string.replace(u'\xa0', '').split(' ', 1)[1]
                if (len(sal) == 1) and (len(re.findall(r'[-]', sal[0].string)) == 1):
                    min_rate = sal[0].string.replace(u'\xa0', '').split('-', 1)[0]
                    max_rate = sal[0].string.replace(u'\xa0', '').split('-', 1)[1]
                df = df.append([{'VacancyName': name, 'Link': link2, 'Min Rate': min_rate, 'Max Rate': max_rate}],
                               ignore_index=True)

    return df[['VacancyName', 'Min Rate', 'Max Rate', 'Link']];

In [15]:
get_pages()

,VacancyName,Min Rate,Max Rate,Link
0,Аналитик отдела маркетинга,35000 руб.,,https://hh.ru/vacancy/33583998?query=%D0%90%D0...
1,Руководитель отдела коммерческой недвижимости,120000 руб.,,https://hh.ru/vacancy/32711614?query=%D0%90%D0...
2,Quantitative Analyst,250000 руб.,,https://hh.ru/vacancy/33543317?query=%D0%90%D0...
3,Quantitative Analyst,250000 руб.,,https://hh.ru/vacancy/33543291?query=%D0%90%D0...
4,Аналитик данных,60000,90000 руб.,https://hh.ru/vacancy/33508837?query=%D0%90%D0...
5,Продуктовый Аналитик / Product Analyst,120000,180000 руб.,https://hh.ru/vacancy/33308536?query=%D0%90%D0...
6,Маркетолог-аналитик,200000 KZT,,https://hh.ru/vacancy/33481528?query=%D0%90%D0...
7,Региональный управляющий магазинами,,120000 руб.,https://hh.ru/vacancy/30308645?query=%D0%90%D0...
8,Системный аналитик,140000 руб.,,https://hh.ru/vacancy/33253710?query=%D0%90%D0...
9,Аналитик 1С,50000 руб.,,https://hh.ru/vacancy/33579014?query=%D0%90%D0...
